In [6]:
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.cross_validation import train_test_split
from pandas import set_option
from sklearn.metrics import recall_score, make_scorer, accuracy_score, precision_score,classification_report, confusion_matrix, roc_curve, roc_auc_score,auc

In [7]:
set_option('display.max_row', 10)
set_option('display.width', 100)
set_option('precision', 2)
set_option('display.max_column', 40)

In [8]:
df_korban = pd.read_csv('datafraud.csv') 
df_korban_fraud = df_korban[df_korban.LABEL==1].copy()
df_non_korban = pd.read_csv('datanonfraud.csv')
frames= [df_korban_fraud, df_non_korban.sample(8821)] # SAMPEL TERDIRI DARI 10000 BARIS YANG DIAMBIL SECARA ACAK
data= pd.concat(frames)
y_true = data['LABEL']
data.head()
y_true=data['LABEL']

C:\Users\leno\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (2,11) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [9]:
data[data['LABEL']==1].describe()    # Describing the data

,LABEL,fiturid,avlblnc,txamount,transtime,seq.trxday,CekSaldo.before,diff.time,diff.txamount,diff.txamount.trx,diff.txamount.bef,percent_saldo
count,1179.0,1.18e+03,1.18e+03,1.18e+03,1179.00,1179.00,1179.00,1179.00,1.18e+03,1.18e+03,1.18e+03,1.18e+03
mean,1.0,1.13e+08,2.23e+07,2.44e+06,42452.84,3.21,0.49,64.25,3.45e+04,7.54e+05,3.45e+04,4.84e+01
std,0.0,3.04e+07,1.89e+08,9.12e+06,20143.72,3.27,0.50,626.61,6.83e+05,1.56e+07,6.83e+05,1.49e+03
min,1.0,1.03e+08,0.00e+00,1.00e+03,8.00,1.00,0.00,0.00,0.00e+00,0.00e+00,0.00e+00,4.08e-05
25%,1.0,1.03e+08,0.00e+00,4.00e+05,27799.00,1.00,0.00,0.00,0.00e+00,0.00e+00,0.00e+00,9.62e-02
50%,1.0,1.03e+08,7.23e+05,1.25e+06,41328.00,2.00,0.00,25.00,6.13e-01,9.09e-01,3.33e-01,3.84e-01
75%,1.0,1.03e+08,6.56e+06,2.00e+06,59873.00,4.00,1.00,34.00,2.04e+00,2.53e+00,9.50e-01,9.04e-01
max,1.0,2.06e+08,2.87e+09,2.65e+08,85168.00,32.00,1.00,16804.00,1.50e+07,5.23e+08,1.50e+07,5.10e+04


In [10]:
data.shape

(10000, 19)

In [11]:
data.head()

,LABEL,fiturid,responcode,avlblnc,txamount,transtime,terminaltype,jenis_kartu,groupname,dana_keluar,lokasi,responcode_before,seq.trxday,CekSaldo.before,diff.time,diff.txamount,diff.txamount.trx,diff.txamount.bef,percent_saldo
3853,1,103010020,0,1.15e+06,100000,28893,ATM_BRI,BRITAMA,TarikTunai,True,dalam,0,1,1,0,0.0,0.0,0.86,0.08
8023,1,103010020,51,0.00e+00,200000,4099,ATM_BRI,BRITAMA,TarikTunai,True,dalam,0,1,0,0,0.0,0.0,0.60,3.83
8024,1,103010020,0,5.23e+04,150000,4123,ATM_BRI,BRITAMA,TarikTunai,True,dalam,51,2,0,24,0.0,0.0,0.25,6.70
9960,1,103010020,0,5.00e+05,1000000,48153,ATM_BRI,SIMPEDES,TarikTunai,True,dalam,0,1,0,0,0.0,0.0,0.33,2.00
10485,1,103010020,0,4.59e+07,1000000,69395,ATM_BRI,BRITAMA,TarikTunai,True,dalam,0,1,0,0,0.0,0.0,0.60,0.02


In [12]:
data.duplicated().sum()

2

In [13]:
data.loc[data.duplicated(keep = False), :] 

,LABEL,fiturid,responcode,avlblnc,txamount,transtime,terminaltype,jenis_kartu,groupname,dana_keluar,lokasi,responcode_before,seq.trxday,CekSaldo.before,diff.time,diff.txamount,diff.txamount.trx,diff.txamount.bef,percent_saldo
269433,0,201010171,0,443886.0,600000,71073,ATM MERAH PUTIH,BRITAMA,CekSaldo,False,dalam,0,0,1,0,0.0,0.0,600000.0,0.0
269458,0,201010171,0,443886.0,600000,71073,ATM MERAH PUTIH,BRITAMA,CekSaldo,False,dalam,0,0,1,0,0.0,0.0,600000.0,0.0
27656,0,201010170,0,61724.0,0,40661,ATM_BRI,BRITAMA,CekSaldo,False,dalam,0,0,1,0,0.0,0.0,0.0,0.0
28178,0,201010170,0,61724.0,0,40661,ATM_BRI,BRITAMA,CekSaldo,False,dalam,0,0,1,0,0.0,0.0,0.0,0.0


In [14]:
data.drop_duplicates(keep= 'first').shape

(9998, 19)

In [15]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10000 entries, 3853 to 72063
Data columns (total 19 columns):
LABEL                10000 non-null int64
fiturid              10000 non-null int64
responcode           9999 non-null object
avlblnc              10000 non-null float64
txamount             10000 non-null int64
transtime            10000 non-null int64
terminaltype         10000 non-null object
jenis_kartu          10000 non-null object
groupname            10000 non-null object
dana_keluar          10000 non-null bool
lokasi               10000 non-null object
responcode_before    10000 non-null object
seq.trxday           10000 non-null int64
CekSaldo.before      10000 non-null int64
diff.time            10000 non-null int64
diff.txamount        10000 non-null float64
diff.txamount.trx    10000 non-null float64
diff.txamount.bef    10000 non-null float64
percent_saldo        10000 non-null float64
dtypes: bool(1), float64(5), int64(7), object(6)
memory usage: 1.5+ MB


In [16]:
data.responcode =pd.to_numeric(data.responcode, errors='coerce').fillna(0, downcast='infer') # TERDAPAT NAN YANG HARUS DIHILANGKAN
data.responcode_before =pd.to_numeric(data.responcode_before, errors='coerce').fillna(0, downcast='infer')

In [17]:
data[data['LABEL']==1].describe()  

,LABEL,fiturid,responcode,avlblnc,txamount,transtime,responcode_before,seq.trxday,CekSaldo.before,diff.time,diff.txamount,diff.txamount.trx,diff.txamount.bef,percent_saldo
count,1179.0,1.18e+03,1179.00,1.18e+03,1.18e+03,1179.00,1179.0,1179.00,1179.00,1179.00,1.18e+03,1.18e+03,1.18e+03,1.18e+03
mean,1.0,1.13e+08,14.80,2.23e+07,2.44e+06,42452.84,12.0,3.21,0.49,64.25,3.45e+04,7.54e+05,3.45e+04,4.84e+01
std,0.0,3.04e+07,25.06,1.89e+08,9.12e+06,20143.72,23.2,3.27,0.50,626.61,6.83e+05,1.56e+07,6.83e+05,1.49e+03
min,1.0,1.03e+08,0.00,0.00e+00,1.00e+03,8.00,0.0,1.00,0.00,0.00,0.00e+00,0.00e+00,0.00e+00,4.08e-05
25%,1.0,1.03e+08,0.00,0.00e+00,4.00e+05,27799.00,0.0,1.00,0.00,0.00,0.00e+00,0.00e+00,0.00e+00,9.62e-02
50%,1.0,1.03e+08,0.00,7.23e+05,1.25e+06,41328.00,0.0,2.00,0.00,25.00,6.13e-01,9.09e-01,3.33e-01,3.84e-01
75%,1.0,1.03e+08,51.00,6.56e+06,2.00e+06,59873.00,0.0,4.00,1.00,34.00,2.04e+00,2.53e+00,9.50e-01,9.04e-01
max,1.0,2.06e+08,93.00,2.87e+09,2.65e+08,85168.00,61.0,32.00,1.00,16804.00,1.50e+07,5.23e+08,1.50e+07,5.10e+04


In [18]:
data= data.dropna()

In [19]:
one_hot = pd.get_dummies(data['responcode']) # RESPONCODE MEMILIKI TYPE OBJECT SEHINGGA HARUS DILAKUKAN ONE HOT ENCODING 
data= data.drop('responcode', axis =1)
data= pd.concat([data, one_hot], axis=1)
data

,LABEL,fiturid,avlblnc,txamount,transtime,terminaltype,jenis_kartu,groupname,dana_keluar,lokasi,responcode_before,seq.trxday,CekSaldo.before,diff.time,diff.txamount,diff.txamount.trx,diff.txamount.bef,percent_saldo,0,3,30,51,61,92,93
3853,1,103010020,1.15e+06,100000,28893,ATM_BRI,BRITAMA,TarikTunai,True,dalam,0,1,1,0,0.00,0.00,0.86,0.08,1,0,0,0,0,0,0
8023,1,103010020,0.00e+00,200000,4099,ATM_BRI,BRITAMA,TarikTunai,True,dalam,0,1,0,0,0.00,0.00,0.60,3.83,0,0,0,1,0,0,0
8024,1,103010020,5.23e+04,150000,4123,ATM_BRI,BRITAMA,TarikTunai,True,dalam,51,2,0,24,0.00,0.00,0.25,6.70,1,0,0,0,0,0,0
9960,1,103010020,5.00e+05,1000000,48153,ATM_BRI,SIMPEDES,TarikTunai,True,dalam,0,1,0,0,0.00,0.00,0.33,2.00,1,0,0,0,0,0,0
10485,1,103010020,4.59e+07,1000000,69395,ATM_BRI,BRITAMA,TarikTunai,True,dalam,0,1,0,0,0.00,0.00,0.60,0.02,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64983,0,201010170,8.40e+05,0,25037,INTERNET BANKING,BRITAMA,CekSaldo,False,dalam,0,0,1,99,0.00,0.00,0.00,0.00,1,0,0,0,0,0,0
222993,0,103010020,6.18e+04,50000,67877,ATM_BRI,BRITAMA,TarikTunai,True,dalam,51,2,1,1325,0.00,0.00,0.00,1.01,1,0,0,0,0,0,0
105040,0,201010170,8.60e+04,0,36192,EDC,SIMPEDES,CekSaldo,False,dalam,0,0,1,949,0.00,0.00,0.00,0.00,1,0,0,0,0,0,0
214496,0,205010360,0.00e+00,101000,77586,INTERNET BANKING,BRITAMA,TransferBRI,True,dalam,0,72,1,202,67.43,67.43,0.66,4.65,1,0,0,0,0,0,0


In [20]:
one_hot = pd.get_dummies(data['responcode_before'])# RESPONCODE_BEFORE MEMILIKI TYPE OBJECT SEHINGGA HARUS DILAKUKAN ONE HOT ENCODING 
data= data.drop('responcode_before', axis =1)
data= pd.concat([data, one_hot], axis=1)


In [21]:
data.head()

,LABEL,fiturid,avlblnc,txamount,transtime,terminaltype,jenis_kartu,groupname,dana_keluar,lokasi,seq.trxday,CekSaldo.before,diff.time,diff.txamount,diff.txamount.trx,diff.txamount.bef,percent_saldo,0,3,30,51,61,92,93,0,3,30,51,53,61
3853,1,103010020,1.15e+06,100000,28893,ATM_BRI,BRITAMA,TarikTunai,True,dalam,1,1,0,0.0,0.0,0.86,0.08,1,0,0,0,0,0,0,1,0,0,0,0,0
8023,1,103010020,0.00e+00,200000,4099,ATM_BRI,BRITAMA,TarikTunai,True,dalam,1,0,0,0.0,0.0,0.60,3.83,0,0,0,1,0,0,0,1,0,0,0,0,0
8024,1,103010020,5.23e+04,150000,4123,ATM_BRI,BRITAMA,TarikTunai,True,dalam,2,0,24,0.0,0.0,0.25,6.70,1,0,0,0,0,0,0,0,0,0,1,0,0
9960,1,103010020,5.00e+05,1000000,48153,ATM_BRI,SIMPEDES,TarikTunai,True,dalam,1,0,0,0.0,0.0,0.33,2.00,1,0,0,0,0,0,0,1,0,0,0,0,0
10485,1,103010020,4.59e+07,1000000,69395,ATM_BRI,BRITAMA,TarikTunai,True,dalam,1,0,0,0.0,0.0,0.60,0.02,1,0,0,0,0,0,0,1,0,0,0,0,0


In [22]:
one_hot = pd.get_dummies(data['terminaltype'])
data= data.drop('terminaltype', axis =1)
data= pd.concat([data, one_hot], axis=1)
data

,LABEL,fiturid,avlblnc,txamount,transtime,jenis_kartu,groupname,dana_keluar,lokasi,seq.trxday,CekSaldo.before,diff.time,diff.txamount,diff.txamount.trx,diff.txamount.bef,percent_saldo,0,3,30,51,61,92,93,0,3,30,51,53,61,ATM MERAH PUTIH,ATM_BRI,Bersama,CDM,EDC,Himbara,INTERNET BANKING,Maestro,Prima,SMS BANKING
3853,1,103010020,1.15e+06,100000,28893,BRITAMA,TarikTunai,True,dalam,1,1,0,0.00,0.00,0.86,0.08,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
8023,1,103010020,0.00e+00,200000,4099,BRITAMA,TarikTunai,True,dalam,1,0,0,0.00,0.00,0.60,3.83,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
8024,1,103010020,5.23e+04,150000,4123,BRITAMA,TarikTunai,True,dalam,2,0,24,0.00,0.00,0.25,6.70,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0
9960,1,103010020,5.00e+05,1000000,48153,SIMPEDES,TarikTunai,True,dalam,1,0,0,0.00,0.00,0.33,2.00,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
10485,1,103010020,4.59e+07,1000000,69395,BRITAMA,TarikTunai,True,dalam,1,0,0,0.00,0.00,0.60,0.02,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64983,0,201010170,8.40e+05,0,25037,BRITAMA,CekSaldo,False,dalam,0,1,99,0.00,0.00,0.00,0.00,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
222993,0,103010020,6.18e+04,50000,67877,BRITAMA,TarikTunai,True,dalam,2,1,1325,0.00,0.00,0.00,1.01,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0
105040,0,201010170,8.60e+04,0,36192,SIMPEDES,CekSaldo,False,dalam,0,1,949,0.00,0.00,0.00,0.00,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
214496,0,205010360,0.00e+00,101000,77586,BRITAMA,TransferBRI,True,dalam,72,1,202,67.43,67.43,0.66,4.65,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0


In [23]:
one_hot = pd.get_dummies(data['jenis_kartu'])
data= data.drop('jenis_kartu', axis =1)
data= pd.concat([data, one_hot], axis=1)
data

,LABEL,fiturid,avlblnc,txamount,transtime,groupname,dana_keluar,lokasi,seq.trxday,CekSaldo.before,diff.time,diff.txamount,diff.txamount.trx,diff.txamount.bef,percent_saldo,0,3,30,51,61,...,93,0,3,30,51,53,61,ATM MERAH PUTIH,ATM_BRI,Bersama,CDM,EDC,Himbara,INTERNET BANKING,Maestro,Prima,SMS BANKING,BRITAMA,PRIORITAS,SIMPEDES
3853,1,103010020,1.15e+06,100000,28893,TarikTunai,True,dalam,1,1,0,0.00,0.00,0.86,0.08,1,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0
8023,1,103010020,0.00e+00,200000,4099,TarikTunai,True,dalam,1,0,0,0.00,0.00,0.60,3.83,0,0,0,1,0,...,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0
8024,1,103010020,5.23e+04,150000,4123,TarikTunai,True,dalam,2,0,24,0.00,0.00,0.25,6.70,1,0,0,0,0,...,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0
9960,1,103010020,5.00e+05,1000000,48153,TarikTunai,True,dalam,1,0,0,0.00,0.00,0.33,2.00,1,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1
10485,1,103010020,4.59e+07,1000000,69395,TarikTunai,True,dalam,1,0,0,0.00,0.00,0.60,0.02,1,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64983,0,201010170,8.40e+05,0,25037,CekSaldo,False,dalam,0,1,99,0.00,0.00,0.00,0.00,1,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0
222993,0,103010020,6.18e+04,50000,67877,TarikTunai,True,dalam,2,1,1325,0.00,0.00,0.00,1.01,1,0,0,0,0,...,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0
105040,0,201010170,8.60e+04,0,36192,CekSaldo,False,dalam,0,1,949,0.00,0.00,0.00,0.00,1,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1
214496,0,205010360,0.00e+00,101000,77586,TransferBRI,True,dalam,72,1,202,67.43,67.43,0.66,4.65,1,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0


In [24]:
one_hot = pd.get_dummies(data['groupname'])
data= data.drop('groupname', axis =1)
data= pd.concat([data, one_hot], axis=1)
data

,LABEL,fiturid,avlblnc,txamount,transtime,dana_keluar,lokasi,seq.trxday,CekSaldo.before,diff.time,diff.txamount,diff.txamount.trx,diff.txamount.bef,percent_saldo,0,3,30,51,61,92,...,53,61,ATM MERAH PUTIH,ATM_BRI,Bersama,CDM,EDC,Himbara,INTERNET BANKING,Maestro,Prima,SMS BANKING,BRITAMA,PRIORITAS,SIMPEDES,CekSaldo,TarikTunai,TransferBRI,TransferBRIVA,TransferBersama
3853,1,103010020,1.15e+06,100000,28893,True,dalam,1,1,0,0.00,0.00,0.86,0.08,1,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0
8023,1,103010020,0.00e+00,200000,4099,True,dalam,1,0,0,0.00,0.00,0.60,3.83,0,0,0,1,0,0,...,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0
8024,1,103010020,5.23e+04,150000,4123,True,dalam,2,0,24,0.00,0.00,0.25,6.70,1,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0
9960,1,103010020,5.00e+05,1000000,48153,True,dalam,1,0,0,0.00,0.00,0.33,2.00,1,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0
10485,1,103010020,4.59e+07,1000000,69395,True,dalam,1,0,0,0.00,0.00,0.60,0.02,1,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64983,0,201010170,8.40e+05,0,25037,False,dalam,0,1,99,0.00,0.00,0.00,0.00,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0
222993,0,103010020,6.18e+04,50000,67877,True,dalam,2,1,1325,0.00,0.00,0.00,1.01,1,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0
105040,0,201010170,8.60e+04,0,36192,False,dalam,0,1,949,0.00,0.00,0.00,0.00,1,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,0,0,0,0
214496,0,205010360,0.00e+00,101000,77586,True,dalam,72,1,202,67.43,67.43,0.66,4.65,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0


In [25]:
one_hot = pd.get_dummies(data['dana_keluar'])
data= data.drop('dana_keluar', axis =1)
data= pd.concat([data, one_hot], axis=1)
data

,LABEL,fiturid,avlblnc,txamount,transtime,lokasi,seq.trxday,CekSaldo.before,diff.time,diff.txamount,diff.txamount.trx,diff.txamount.bef,percent_saldo,0,3,30,51,61,92,93,...,ATM MERAH PUTIH,ATM_BRI,Bersama,CDM,EDC,Himbara,INTERNET BANKING,Maestro,Prima,SMS BANKING,BRITAMA,PRIORITAS,SIMPEDES,CekSaldo,TarikTunai,TransferBRI,TransferBRIVA,TransferBersama,False,True
3853,1,103010020,1.15e+06,100000,28893,dalam,1,1,0,0.00,0.00,0.86,0.08,1,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1
8023,1,103010020,0.00e+00,200000,4099,dalam,1,0,0,0.00,0.00,0.60,3.83,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1
8024,1,103010020,5.23e+04,150000,4123,dalam,2,0,24,0.00,0.00,0.25,6.70,1,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1
9960,1,103010020,5.00e+05,1000000,48153,dalam,1,0,0,0.00,0.00,0.33,2.00,1,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1
10485,1,103010020,4.59e+07,1000000,69395,dalam,1,0,0,0.00,0.00,0.60,0.02,1,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64983,0,201010170,8.40e+05,0,25037,dalam,0,1,99,0.00,0.00,0.00,0.00,1,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,1,0
222993,0,103010020,6.18e+04,50000,67877,dalam,2,1,1325,0.00,0.00,0.00,1.01,1,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1
105040,0,201010170,8.60e+04,0,36192,dalam,0,1,949,0.00,0.00,0.00,0.00,1,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0,0,0,1,1,0,0,0,0,1,0
214496,0,205010360,0.00e+00,101000,77586,dalam,72,1,202,67.43,67.43,0.66,4.65,1,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,1


In [26]:
one_hot = pd.get_dummies(data['lokasi'])
data= data.drop('lokasi', axis =1)
data= pd.concat([data, one_hot], axis=1)
data

,LABEL,fiturid,avlblnc,txamount,transtime,seq.trxday,CekSaldo.before,diff.time,diff.txamount,diff.txamount.trx,diff.txamount.bef,percent_saldo,0,3,30,51,61,92,93,0,...,Bersama,CDM,EDC,Himbara,INTERNET BANKING,Maestro,Prima,SMS BANKING,BRITAMA,PRIORITAS,SIMPEDES,CekSaldo,TarikTunai,TransferBRI,TransferBRIVA,TransferBersama,False,True,dalam,luar
3853,1,103010020,1.15e+06,100000,28893,1,1,0,0.00,0.00,0.86,0.08,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,1,0
8023,1,103010020,0.00e+00,200000,4099,1,0,0,0.00,0.00,0.60,3.83,0,0,0,1,0,0,0,1,...,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,1,0
8024,1,103010020,5.23e+04,150000,4123,2,0,24,0.00,0.00,0.25,6.70,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,1,0
9960,1,103010020,5.00e+05,1000000,48153,1,0,0,0.00,0.00,0.33,2.00,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1,1,0
10485,1,103010020,4.59e+07,1000000,69395,1,0,0,0.00,0.00,0.60,0.02,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64983,0,201010170,8.40e+05,0,25037,0,1,99,0.00,0.00,0.00,0.00,1,0,0,0,0,0,0,1,...,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,1,0,1,0
222993,0,103010020,6.18e+04,50000,67877,2,1,1325,0.00,0.00,0.00,1.01,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,1,0
105040,0,201010170,8.60e+04,0,36192,0,1,949,0.00,0.00,0.00,0.00,1,0,0,0,0,0,0,1,...,0,0,1,0,0,0,0,0,0,0,1,1,0,0,0,0,1,0,1,0
214496,0,205010360,0.00e+00,101000,77586,72,1,202,67.43,67.43,0.66,4.65,1,0,0,0,0,0,0,1,...,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,1,1,0


In [27]:
X = data.iloc[:, data.columns != "LABEL"]
Y = data.iloc[:, data.columns == "LABEL"]

In [28]:
X.head()


,fiturid,avlblnc,txamount,transtime,seq.trxday,CekSaldo.before,diff.time,diff.txamount,diff.txamount.trx,diff.txamount.bef,percent_saldo,0,3,30,51,61,92,93,0,3,...,Bersama,CDM,EDC,Himbara,INTERNET BANKING,Maestro,Prima,SMS BANKING,BRITAMA,PRIORITAS,SIMPEDES,CekSaldo,TarikTunai,TransferBRI,TransferBRIVA,TransferBersama,False,True,dalam,luar
3853,103010020,1.15e+06,100000,28893,1,1,0,0.0,0.0,0.86,0.08,1,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,1,0
8023,103010020,0.00e+00,200000,4099,1,0,0,0.0,0.0,0.60,3.83,0,0,0,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,1,0
8024,103010020,5.23e+04,150000,4123,2,0,24,0.0,0.0,0.25,6.70,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,1,0
9960,103010020,5.00e+05,1000000,48153,1,0,0,0.0,0.0,0.33,2.00,1,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1,1,0
10485,103010020,4.59e+07,1000000,69395,1,0,0,0.0,0.0,0.60,0.02,1,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,1,0


In [29]:
from sklearn.model_selection import train_test_split,GridSearchCV, StratifiedKFold
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.37, random_state= None) # UKURAN DATA TESTING YAKNI 2000 BARIS

In [30]:
# STANDARDISASI NILAI PADA SETIAP FITUR
from sklearn.preprocessing import StandardScaler
stdsc = StandardScaler()
X_train_std = stdsc.fit_transform(X_train)
X_test_std = stdsc.transform(X_test)

In [31]:
print(X_train_std)

[[ 0.47876898 -0.11241467 -0.17862387 ...  0.71266999  0.16142686
  -0.16142686]
 [ 0.37013618 -0.10968811 -0.22097793 ... -1.403174    0.16142686
  -0.16142686]
 [ 0.47876898 -0.11241467 -0.06215018 ...  0.71266999  0.16142686
  -0.16142686]
 ...
 [ 0.47876898 -0.11241467 -0.20592953 ...  0.71266999  0.16142686
  -0.16142686]
 [ 0.47876898 -0.11241467 -0.21420128 ...  0.71266999  0.16142686
  -0.16142686]
 [ 0.47876898 -0.11241467 -0.22076616 ...  0.71266999  0.16142686
  -0.16142686]]


In [32]:
# FITUR ID SUDAH DIWALIKI OLEH GROUPNAME
data.drop(['fiturid', 'dalam', 'luar', 'false', 'true'], axis=1, inplace=True)
data.head()

,LABEL,avlblnc,txamount,transtime,seq.trxday,CekSaldo.before,diff.time,diff.txamount,diff.txamount.trx,diff.txamount.bef,percent_saldo,0,3,30,51,61,92,93,0,3,...,ATM MERAH PUTIH,ATM_BRI,Bersama,CDM,EDC,Himbara,INTERNET BANKING,Maestro,Prima,SMS BANKING,BRITAMA,PRIORITAS,SIMPEDES,CekSaldo,TarikTunai,TransferBRI,TransferBRIVA,TransferBersama,False,True
3853,1,1.15e+06,100000,28893,1,1,0,0.0,0.0,0.86,0.08,1,0,0,0,0,0,0,1,0,...,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1
8023,1,0.00e+00,200000,4099,1,0,0,0.0,0.0,0.60,3.83,0,0,0,1,0,0,0,1,0,...,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1
8024,1,5.23e+04,150000,4123,2,0,24,0.0,0.0,0.25,6.70,1,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1
9960,1,5.00e+05,1000000,48153,1,0,0,0.0,0.0,0.33,2.00,1,0,0,0,0,0,0,1,0,...,0,1,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1
10485,1,4.59e+07,1000000,69395,1,0,0,0.0,0.0,0.60,0.02,1,0,0,0,0,0,0,1,0,...,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1


In [35]:
model1 = xgb.XGBClassifier()
model2 = xgb.XGBClassifier(n_estimators=100, max_depth=8, learning_rate=0.1, subsample=0.5)

train_model1 = model1.fit(X_train_std, y_train.values.reshape(-1,))
train_model2 = model2.fit(X_train_std, y_train.values.reshape(-1,))

In [38]:
from sklearn.metrics import classification_report

pred1 = train_model1.predict(X_test_std)
pred2 = train_model2.predict(X_test_std)

print(classification_report(y_test, pred1))
print(classification_report(y_test, pred1))

             precision    recall  f1-score   support

          0       0.98      0.99      0.98      3277
          1       0.89      0.85      0.87       423

avg / total       0.97      0.97      0.97      3700

             precision    recall  f1-score   support

          0       0.98      0.99      0.98      3277
          1       0.89      0.85      0.87       423

avg / total       0.97      0.97      0.97      3700



C:\Users\leno\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\leno\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [54]:
from sklearn.metrics import recall_score, make_scorer, accuracy_score, precision_score,classification_report, confusion_matrix, roc_curve, roc_auc_score,auc
train_y_pred = model1.predict(X_train_std)
auc = roc_auc_score(y_train, train_y_pred)
print("Performance sur le train : ", auc)

# Performance sur le test
test_y_pred = model1.predict(X_test_std)
auc = roc_auc_score(y_test, test_y_pred)
print("Performance sur le test : ", auc)


train_y_pred = model2.predict(X_train_std)
auc = roc_auc_score(y_train, train_y_pred)
print("Performance sur le train : ", auc)

# Performance sur le test
test_y_pred = model2.predict(X_test_std)
auc = roc_auc_score(y_test, test_y_pred)
print("Performance sur le test : ", auc)


Performance sur le train :  0.9184403559403559
Performance sur le test :  0.9176364243661136
Performance sur le train :  0.9879449254449254
Performance sur le test :  0.9649195517724725


C:\Users\leno\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\leno\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\leno\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\leno\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151

In [39]:
#Let's use accuracy score
from sklearn.metrics import accuracy_score

print("Accuracy for model 1: %.2f" % (accuracy_score(y_test, pred1) * 100))
print("Accuracy for model 2: %.2f" % (accuracy_score(y_test, pred2) * 100))

Accuracy for model 1: 97.08
Accuracy for model 2: 98.89


In [45]:
#Let's do a little Gridsearch, Hyperparameter Tunning
model3 = xgb.XGBClassifier(
 learning_rate =0.1,
 n_estimators=1000,
 max_depth=5,
 min_child_weight=1,
 gamma=0,
 subsample=0.8,
 colsample_bytree=0.8,
 objective= 'binary:logistic',
 nthread=4,
 scale_pos_weight=1,
 seed=27)

In [46]:
train_model3 = model3.fit(X_train_std, y_train.values.reshape(-1,))
pred3 = train_model3.predict(X_test_std)
print("Accuracy for model 3: %.2f" % (accuracy_score(y_test, pred3) * 100))

Accuracy for model 3: 99.30


C:\Users\leno\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [47]:
from sklearn.model_selection import GridSearchCV

param_test = {
 'max_depth':[4,5,6],
 'min_child_weight':[4,5,6]
}
gsearch = GridSearchCV(estimator = xgb.XGBClassifier( learning_rate=0.1, n_estimators=140, max_depth=5,
 min_child_weight=2, gamma=0, subsample=0.8, colsample_bytree=0.8,
 objective= 'binary:logistic', nthread=4, scale_pos_weight=1,seed=27), 
 param_grid = param_test, scoring='roc_auc',n_jobs=4,iid=False, cv=5)

train_model4 = gsearch.fit(X_train_std, y_train.values.reshape(-1,))
pred4 = train_model4.predict(X_test_std)
print("Accuracy for model 4: %.2f" % (accuracy_score(y_test, pred4) * 100))

Accuracy for model 4: 98.84


C:\Users\leno\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [48]:
param_test2b = {
 'min_child_weight':[6,8,10,12]
}
gsearch2b = GridSearchCV(estimator = xgb.XGBClassifier( learning_rate=0.1, n_estimators=140, max_depth=4,
 min_child_weight=2, gamma=0, subsample=0.8, colsample_bytree=0.8,
 objective= 'binary:logistic', nthread=4, scale_pos_weight=1,seed=27), 
 param_grid = param_test2b, scoring='roc_auc',n_jobs=4,iid=False, cv=5)

train_model5 = gsearch2b.fit(X_train_std, y_train.values.reshape(-1,))
pred5 = train_model5.predict(X_test_std)
print("Accuracy for model 5: %.2f" % (accuracy_score(y_test, pred5) * 100))

Accuracy for model 5: 98.68


C:\Users\leno\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [49]:
#Tune Gamma
param_test3 = {
 'gamma':[i/10.0 for i in range(0,5)]
}
gsearch3 = GridSearchCV(estimator = xgb.XGBClassifier( learning_rate =0.1, n_estimators=140, max_depth=4,
 min_child_weight=6, gamma=0, subsample=0.8, colsample_bytree=0.8,
 objective= 'binary:logistic', nthread=4, scale_pos_weight=1,seed=27), 
 param_grid = param_test3, scoring='roc_auc',n_jobs=4,iid=False, cv=5)

train_model6 = gsearch3.fit(X_train_std, y_train.values.reshape(-1,))
pred6 = train_model6.predict(X_test_std)
print("Accuracy for model 6: %.2f" % (accuracy_score(y_test, pred6) * 100))

Accuracy for model 6: 98.70


C:\Users\leno\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [50]:
xgb2 = xgb.XGBClassifier(
 learning_rate =0.7,
 n_estimators=1000,
 max_depth=4,
 min_child_weight=6,
 gamma=0,
 subsample=0.8,
 colsample_bytree=0.8,
 objective= 'binary:logistic',
 nthread=4,
 scale_pos_weight=1,
 seed=27)

train_model7 = xgb2.fit(X_train_std, y_train.values.reshape(-1,))
pred7 = train_model7.predict(X_test_std)
print("Accuracy for model 7: %.2f" % (accuracy_score(y_test, pred7) * 100))

Accuracy for model 7: 99.08


C:\Users\leno\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [51]:
#Let's train a fast RandomForest on the dataset
from sklearn.ensemble import RandomForestClassifier

rfc = RandomForestClassifier()
rfc_model = rfc.fit(X_train_std, y_train.values.reshape(-1,))
pred8 = rfc_model.predict(X_test_std)
print("Accuracy for Random Forest Model: %.2f" % (accuracy_score(y_test, pred8) * 100))

Accuracy for Random Forest Model: 99.05
